In [33]:
import os
from PIL import ImageFilter
import cv2
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.utils import resample
from PIL import Image
from glob import glob
import matplotlib.pyplot as plt

#Se elije una carpeta desde donde se trabaja
mainpathlocation = 'D:/CosasUniversidad/Material/ULA6-Semestre12/Taller-de-Titulo-II/Recursos/HAM10000/HAMTESTS/'
#mainpathlocation = 'C:/Users/malz1/Desktop/2023/tesis/'
os.chdir(mainpathlocation)

In [34]:
#Se lee el csv y se obtienen las clases
metadf = pd.read_csv(mainpathlocation+"HAM10000_metadata.csv")
le = LabelEncoder()
le.fit(metadf['dx'])
LabelEncoder()
print(list(le.classes_))

metadf['label'] = le.transform(metadf["dx"])

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


In [35]:
#Se "enlazan" los datos en el csv con sus imagenes correspondientes
os.chdir(mainpathlocation)
image_path = {os.path.splitext(os.path.basename(x))[0]: x
              for x in glob(os.path.join('FullHAM/','*.jpg'))}
metadf['path'] = metadf['image_id'].map(image_path.get)

Resample (data balancing)

In [36]:
from sklearn.utils import resample
print(metadf['label'].value_counts())

metadf_0 = metadf[metadf['label'] == 0]
metadf_1 = metadf[metadf['label'] == 1]
metadf_2 = metadf[metadf['label'] == 2]
metadf_3 = metadf[metadf['label'] == 3]
metadf_4 = metadf[metadf['label'] == 4]
metadf_5 = metadf[metadf['label'] == 5]
metadf_6 = metadf[metadf['label'] == 6]

#Se equilibran los datos, para evitar un sobreajuste en el entrenamiento
#Se elije la cantidad de datos en cada clase modificando n_samples
n_samples=1500
metadf_0_balanced = resample(metadf_0, replace=True, n_samples=n_samples, random_state=42) 
metadf_1_balanced = resample(metadf_1, replace=True, n_samples=n_samples, random_state=42) 
metadf_2_balanced = resample(metadf_2, replace=True, n_samples=n_samples, random_state=42)
metadf_3_balanced = resample(metadf_3, replace=True, n_samples=n_samples, random_state=42)
metadf_4_balanced = resample(metadf_4, replace=True, n_samples=n_samples, random_state=42)
metadf_5_balanced = resample(metadf_5, replace=True, n_samples=n_samples, random_state=42)
metadf_6_balanced = resample(metadf_6, replace=True, n_samples=n_samples, random_state=42)

metadf = pd.concat([metadf_0_balanced, metadf_1_balanced, 
                              metadf_2_balanced, metadf_3_balanced, 
                              metadf_4_balanced, metadf_5_balanced, metadf_6_balanced])

#Balanced classes
print(metadf['label'].value_counts())

label
5    6705
4    1113
2    1099
1     514
0     327
6     142
3     115
Name: count, dtype: int64
label
0    500
1    500
2    500
3    500
4    500
5    500
6    500
Name: count, dtype: int64


Cropping machine

In [37]:
#Se define una función que corta las imagenes centrando la imagen en el centro de la lesion
def cropImageFromCenter(centerX,centerY,ogWidth,ogHeight,ratio):
    croppedWidth = ogWidth*ratio
    croppedHeight = ogHeight*ratio

    if centerX - croppedWidth/2 < 0:
      croppedA = 0
      croppedB = centerX + croppedWidth/2 + (croppedWidth/2-centerX)
    elif centerX + croppedWidth/2 > ogWidth:
      croppedA = centerX - croppedWidth/2 - ((croppedWidth/2+centerX)-ogWidth)
      croppedB = ogWidth
    else:
      croppedA = centerX - croppedWidth/2
      croppedB = centerX + croppedWidth/2

    if centerY-croppedHeight/2 < 0:
      croppedC = 0
      croppedD = centerY + croppedHeight/2 + (croppedHeight/2-centerY)
    elif centerY+croppedHeight/2 > ogHeight:
      croppedC = centerY - croppedHeight/2 - ((croppedHeight/2+centerY)-ogHeight)
      croppedD = ogHeight
    else:
      croppedC = centerY - croppedHeight/2
      croppedD = centerY + croppedHeight/2

    return croppedA,croppedB,croppedC,croppedD

Métodos

In [38]:
#Funcion que elimina los pelos de la imagen
def dullRazor(path): 
    image=cv2.imread(path,cv2.IMREAD_COLOR)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY )
    kernel = cv2.getStructuringElement(1,(9,9))
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    bhg = cv2.GaussianBlur(blackhat,(3,3),cv2.BORDER_DEFAULT)
    ret,mask = cv2.threshold(bhg,10,255,cv2.THRESH_BINARY)
    dst = cv2.inpaint(image,mask,6,cv2.INPAINT_TELEA)
    return dst
#devuelve a color

#Se elimina el ruido
def medianFilter(image):
    img = cv2.medianBlur(image, 3)
    return img

#Segmenta la imagen en blanco y negro
def segment(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(img, 125, 255, cv2.THRESH_BINARY_INV)
    return thresh1

#encuentra el centro de la herida y corta usando la funcion de corte definida anteriormente
def center_crop(image):
    h,w = image.shape
    imageMoments = cv2.moments(image)
    if imageMoments['m00'] == 0:
        cx = w/2
        cy = h/2
    else:
        cx = int(imageMoments['m10']/imageMoments['m00'])
        cy = int(imageMoments['m01']/imageMoments['m00'])
    croppedA,croppedB,croppedC,croppedD = cropImageFromCenter(cx,cy,w,h,0.9)
    image = image[int(croppedC):int(croppedD), int(croppedA):int(croppedB)]
    return image
    
def preProcess(x):
    x = dullRazor(x)
    x = medianFilter(x)
    x = segment(x)
    x = center_crop(x)
    return x

In [ ]:
#A continuación se separan las funciones de entrenamiento por tipo de dato
#Así, se puede entrenar con solo lo que se requiera, y se guarda en un archivo .pkl para uso en svmTraining

Asimetria

In [39]:

def getAsimetria(image):
    h, w = image.shape
    if h > w:
        image = image[int((h-w)/2):int(h-((h-w)/2)), int(0): int(w)]
    elif w > h:
        image = image[int(0):int(h), int((w-h)/2): int(w-((w-h)/2))]    
    else:
        image
    X_Hist = cv2.calcHist(image, [0], None, [256], [0,256], False)
    Y_Hist = cv2.calcHist(image, [157], None, [256], [0,256], False)
    compare_val = cv2.compareHist(X_Hist ,Y_Hist,cv2.HISTCMP_INTERSECT)
    #print(compare_val)
    return compare_val
metadf['Asimetria'] = metadf['path'].map(lambda x: np.asarray( getAsimetria(preProcess(x))))

In [40]:
metadf.to_pickle("svmDataset.pkl")

Circularidad

In [41]:
#os.chdir(mainpathlocation)
#DEBE RECIBIR GRAYSCALE
def getCircularidad(image):
    imageMoments = cv2.moments(image)
    area = imageMoments['m00']
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    perimeter = 0
    for coords in contours:
      perimeter = perimeter + cv2.arcLength(coords, False)
    if perimeter == 0:
      perimeter = 1
    circularity = (4*area*math.pi)/perimeter*perimeter
    return circularity
    

metadf['Circularidad'] = metadf['path'].map(lambda x: np.asarray( getCircularidad(preProcess(x)) ))

In [42]:
metadf.to_pickle("svmDataset.pkl")

In [43]:
def getThinRatio(img):
    imageMoments = cv2.moments(img)
    area = imageMoments['m00']
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    perimeter = 0
    for coords in contours:
      perimeter = perimeter + cv2.arcLength(coords, False)
    if perimeter == 0:
      perimeter = 1
    return 4*math.pi*(area/(perimeter*perimeter))

metadf['ThinRatio'] = metadf['path'].map(lambda x: np.asarray( getThinRatio(preProcess(x)) ))

In [44]:
metadf.to_pickle("svmDataset.pkl")

In [45]:
def getAspectRatio(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    xA=[] 
    yA=[] 
    w=0
    h=0
    for coords in contours:
        x2,y2,w2,h2 = cv2.boundingRect(coords)
        xA.append(x2)
        yA.append(y2)
    
    if (len(xA) != 0) and (len(yA) != 0):
        x = xA[np.argmin(xA)]
        y = yA[np.argmin(yA)]
        w = xA[np.argmax(xA)] - x
        h = yA[np.argmax(yA)] - y
    if h == 0:
        h = 1
    return (w/h)

metadf['AspectRatio'] = metadf['path'].map(lambda x: np.asarray( getAspectRatio(preProcess(x)) ))

In [46]:
metadf['AvgRed'] = metadf['path'].map(lambda x: np.asarray( np.mean(medianFilter(dullRazor(x))[:,:,0])) )

In [47]:
metadf['AvgGreen'] = metadf['path'].map(lambda x: np.asarray( np.mean(medianFilter(dullRazor(x))[:,:,1])) )

In [48]:
metadf['AvgBlue'] = metadf['path'].map(lambda x: np.asarray( np.mean(medianFilter(dullRazor(x))[:,:,2])) )

In [49]:
metadf.to_pickle("svmDataset.pkl")

In [50]:
metadf['AvgHue'] = metadf['path'].map(lambda x: np.asarray( np.mean(cv2.cvtColor(medianFilter(dullRazor(x)), cv2.COLOR_RGB2HSV)[:,:,0]) ) )

In [51]:
metadf['AvgSaturation'] = metadf['path'].map(lambda x: np.asarray( np.mean(cv2.cvtColor(medianFilter(dullRazor(x)), cv2.COLOR_RGB2HSV)[:,:,1]) ) )

In [52]:
metadf.to_pickle("svmDataset.pkl")

In [53]:
def getDiameter(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    xA=[] 
    yA=[] 
    w=0
    h=0
    for coords in contours:
        x2,y2,w2,h2 = cv2.boundingRect(coords)
        xA.append(x2)
        yA.append(y2)
    
    if (len(xA) != 0) and (len(yA) != 0):
        x = xA[np.argmin(xA)]
        y = yA[np.argmin(yA)]
        w = xA[np.argmax(xA)] - x
        h = yA[np.argmax(yA)] - y
    diametro = 0
    if w>h:
      diametro = w
    else:
      diametro = h
    return diametro

metadf['Diameter'] = metadf['path'].map(lambda x: np.asarray( getDiameter(preProcess(x)) ))

In [54]:
def getIntensityAverage(x):
    
    avgR = np.mean(x[:,0])
    avgG = np.mean(x[:,1])
    avgB = np.mean(x[:,2])
    return (avgR+avgG+avgB)/3


metadf['AvgIntensity'] = metadf['path'].map(lambda x: np.asarray( getIntensityAverage(medianFilter(dullRazor(x))[:,:,0])) )

In [55]:
metadf.to_pickle("svmDataset.pkl")